In [1]:
import os
from dotenv import load_dotenv

# Load AWS credentials from .env file
load_dotenv()

os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY")
os.environ["AWS_REGION"] = os.getenv("AWS_REGION", "us-east-1")


In [55]:
# !uv pip install langchain langchain-community boto3
# !uv pip install langchain-aws
# !uv pip install notion-client

Audited 1 package in 278ms


In [12]:
import boto3
from langchain_aws.embeddings import BedrockEmbeddings

# Initialize the client
client = boto3.client("bedrock-runtime", region_name="us-east-1")
# Use Titan Embeddings V2
titan_embedder = BedrockEmbeddings(
    client=client,
    model_id="amazon.titan-embed-text-v2:0"
)


In [13]:
## dry run 
sample_text = "Airport lounge access is available with Amex Platinum."
embedding = titan_embedder.embed_query(sample_text)

print(f"Embedding size: {len(embedding)}")
print(f"First 5 dimensions: {embedding[:5]}")

Embedding size: 1024
First 5 dimensions: [-0.04292689636349678, 0.06853839755058289, 0.019514024257659912, -0.007966389879584312, 0.0385114811360836]


### Notion Text Handling

- notion_list_dbs.py

In [17]:
import os
from notion_client import Client
from dotenv import load_dotenv

load_dotenv()
notion = Client(auth=os.getenv("NOTION_API_KEY"))

# Search for databases
response = notion.search(filter={"property": "object", "value": "database"})

for result in response["results"]:
    print("Title:", result["title"][0]["plain_text"] if result["title"] else "No title")
    print("ID:", result["id"])
    print("---")


Title: Topics
ID: 2394ba18-200f-81de-8a3e-ee605002d26a
---


- notion_fetch_rows.py


In [18]:
import os
from notion_client import Client
from dotenv import load_dotenv

load_dotenv()
notion = Client(auth=os.getenv("NOTION_API_KEY"))

database_id = "2394ba18-200f-81de-8a3e-ee605002d26a"

# Query database rows (pages)
response = notion.databases.query(database_id=database_id)

for page in response["results"]:
    props = page["properties"]
    title = props["Name"]["title"][0]["plain_text"] if props["Name"]["title"] else "Untitled"
    print("Title:", title)

    # You can extract more fields depending on your DB schema
    for key, val in props.items():
        if val["type"] == "rich_text":
            content = val["rich_text"][0]["plain_text"] if val["rich_text"] else ""
            print(f"{key}: {content}")
    print("---")


Title: 🛒 Consumer Categories
Description: Description
---
Title: Travel-Related Rewards Categories
Description: This page outlines the travel-related categories eligible for additional rewards on your Amex Card, including airfare, hotels, transit, and car rentals.
---
Title: Amex Rewards Information
Description: Here you can find detailed information about Amex Rewards, including points system, redemption options, bonus categories, and expiration rules.
---
Title: Business-Related Rewards Categories
Description: This page outlines business-specific purchase categories that are eligible for additional rewards on your Amex Card, including Amazon Business, shipping, software, office supplies, and wireless providers.
---


notion_get_blocks.py

- Tile content pull code 

In [ ]:
https://www.notion.so/Amex-Help-Center-Knowledge-Base-2394ba18200f80c3ba0ed88f417c09d7?source=copy_link#2394ba18200f802ab396e79e36e9f40d
https://www.notion.so/Amex-Help-Center-Knowledge-Base-2394ba18200f80c3ba0ed88f417c09d7?source=copy_link#23a4ba18200f80f69e00ed436fc647c6
https://www.notion.so/Amex-Help-Center-Knowledge-Base-2394ba18200f80c3ba0ed88f417c09d7?source=copy_link#23a4ba18200f801786d2de2f26bb4a9c
https://www.notion.so/Amex-Help-Center-Knowledge-Base-2394ba18200f80c3ba0ed88f417c09d7?source=copy_link#23a4ba18200f8044b127e83767cb5ef5

- WORKING FOR FAQs

In [80]:
def extract_block_text(block):
    block_type = block["type"]
    rich_parts = block.get(block_type, {}).get("rich_text", [])
    return "".join(part.get("plain_text", "") for part in rich_parts)

def extract_children_recursive(block_id, notion):
    """Recursively fetches children (like nested answers)."""
    children = []
    response = notion.blocks.children.list(block_id)
    for child in response.get("results", []):
        text = extract_block_text(child)
        if text:
            if child["type"] == "bulleted_list_item":
                children.append(f"- {text}")
            else:
                children.append(text)
        # Recurse further if this child has its own children
        if child.get("has_children"):
            children.extend(extract_children_recursive(child["id"], notion))
    return children

def convert_page_to_doc_full(page_title, page_id, notion):
    content_lines = []

    def walk_blocks(blocks):
        for block in blocks:
            block_type = block["type"]
            text = extract_block_text(block)

            if block_type.startswith("heading") and text:
                content_lines.append(f"\n## {text}")
            elif block_type == "paragraph" and text:
                content_lines.append(text)
            elif block_type == "callout" and text:
                content_lines.append(f"💬 {text}")
            elif block_type == "bulleted_list_item" and text:
                content_lines.append(f"- {text}")
            elif block_type == "toggle" and text:
                content_lines.append(f"\n### ❓ {text}")
                # Pull its children recursively
                children = extract_children_recursive(block["id"], notion)
                content_lines.extend(children)

            # Recurse into all child blocks
            if block.get("has_children") and block_type != "toggle":
                sub_blocks = notion.blocks.children.list(block["id"])["results"]
                walk_blocks(sub_blocks)

    # Start walking from root page
    root_blocks = notion.blocks.children.list(page_id)["results"]
    walk_blocks(root_blocks)

    full_text = "\n".join(content_lines).strip()
    return {
        "page_content": full_text,
        "metadata": {"source": page_title}
    }


In [89]:
from assistant.core.notion_amex_faqs import convert_page_to_doc_full
from notion_client import Client
import os

notion = Client(auth=os.getenv("NOTION_API_KEY"))
page_id = "2394ba18-200f-80c3-ba0e-d88f417c09d7"
page_title = "Amex Knowledge Base"

doc = convert_page_to_doc_full(page_title, page_id, notion)

# Preview the dry run
print("🔍 FULL DRY RUN PREVIEW:\n")
for i, line in enumerate(doc["page_content"].splitlines(), 1):
    print(f"{i:02}. {line}")



🔍 FULL DRY RUN PREVIEW:

01. ## Rewards Frequently Asked Questions
02. 
03. ##  What purchases are eligible to earn rewards?
04. Eligible purchases include goods and services minus returns and credits.
05. They do not include:
06. - Fees or interest charges
07. - Balance transfers or cash advances
08. - Peer-to-peer payments
09. - Gift cards or traveler’s checks
10. - Prepaid card reloads
11. - Cash equivalent purchases
12. 
13. ## Where can I see my purchases that qualified for additional rewards?
14. 
15. You can view them under Bonus Rewards Detail in your Amex online account.
16. 
17. ## How do you define a purchase made in the United States?
18. A purchase is considered U.S.-based if the merchant is located and registered in the U.S. Note: Some online/mobile purchases might be processed internationally.
19. 
20. ## Are purchases made outside the United States eligible for additional rewards?
21. It depends on your card’s terms.
22. Check your Cardmember Agreement or Membership Rew

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

chunks = splitter.create_documents([doc["page_content"]])

# Then embed these chunks into FAISS
vectorstore = FAISS.from_documents(chunks, titan_embedder)
vectorstore.save_local("vectorstore/notion_amex_faqs")

In [90]:
# Dry run preview
# Step 1: Load the FAISS vector store
vectorstore = FAISS.load_local(
    "vectorstore/notion_amex_faqs",
    titan_embedder,
    allow_dangerous_deserialization=True
)

# Step 2: Define a test query
query = "How do I earn rewards for booking flights with Amex?"

# Step 3: Perform a similarity search
results = vectorstore.similarity_search(query, k=3)

# Step 4: Preview top matches
print(f"🔍 Query: {query}\n")
for i, doc in enumerate(results, 1):
    print(f"--- Match {i} ---\n{doc.page_content}\n")

🔍 Query: How do I earn rewards for booking flights with Amex?

--- Match 1 ---
## Travel Rewards Frequently Asked Questions

## ✈️ Airfare – Frequently Asked Questions

## Do I earn additional rewards when I book airfare through American Express Travel?
Yes, you will earn additional rewards when you book airfare with American Express Travel — but not if it's part of a vacation package.

--- Match 2 ---
##  How do I earn additional rewards on airfare purchases?
Book a scheduled flight on a passenger airline, and make the purchase directly with the airline.

##  I think I should’ve earned additional rewards. What should I do?
Call the number on the back of your card to request a review of your transaction.

## 🏨 Hotels – Frequently Asked Questions

--- Match 3 ---
## What if I book my hotel as part of a vacation package?

Hotel rooms bundled into vacation packages do not qualify for additional rewards.

##  I think I should’ve earned additional rewards. What should I do?
Contact Amex cus